In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Data

In [ ]:
#Count the classificaition types for prediction
def count(pred):
    pred = list(pred.argmax(axis=1))
    count0, count1, count2, count3 = [0,0,0,0]
    for val in pred:
        if val == 0:
            count0 += 1
        elif val == 1:
            count1 += 1
        elif val == 2:
            count2 += 1
        else:
            count3 += 1
    print("inter labels: " + str(count0)+",", " vertebrae labels: " + str(
        count1)+",", " tailbone labels: " + str(
        count2)+",", " other labels: " + str(count3))

In [ ]:
X_train = np.load('data/X_train_all_newpig.npy')
y_train = np.load('data/labels_newpig.npy', allow_pickle = True)

# Move labels to one list
y_all = []
for aqu in y_train:
    for frame in aqu:
        y_all.append(frame)

#Separate into training and testing
X_train_c = X_train[0:15307]
X_train_cc = X_train_c[0:int(len(X_train_c) * 0.8)+1]
X_test = np.array(X_train_c[12245:-1])
X_train = np.array(X_train_cc)
y_train = np.array(y_all[0:int(len(y_all) * 0.8)+1])
y_test = np.array(y_all[12245:-1])

# one hot encoding
ohe = True
if (ohe == True) and len(y_train.shape) < 2 and len(y_test.shape) < 2:
    y_train = to_categorical(y_train, 3)
    y_test = to_categorical(y_test, 3)

# CNN

In [ ]:
#Define CNN model parameters
depths = [1,2,3,4,5] #depth of CNN model
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5] #Dropout in the fully connected layer
kernel_sizes = [2,3, 4, 5, 6] 
outputs = [1]
dense_width = [1,2,3] #dense layer width
dense_length = [3,5]
batch_sizes = [32]

# Define the input shape
input_shape = (616, 756, 1)
fig_path = 'results/figures/'
for batch in batch_sizes:
    for dropout_rate in dropout_rates:
        for kernel_size in kernel_sizes:
            for output in outputs:
                for dense_w in dense_width:
                    for dense_l in dense_length:
                        for depth in depths:
                            CNN = Sequential(name="Sequential_CNN")
                            CNN.add(Conv2D(128, 
                                           kernel_size=(kernel_size, kernel_size), 
                                           strides=(2, 2), padding="same", 
                                           activation="relu", input_shape=input_shape))
                            out = 128
                            for i in range(depth-1):
                                CNN.add(Conv2D(out, kernel_size=(kernel_size, kernel_size), 
                                               strides=(2, 2), padding="same", activation="relu"))
                                out = int(out/2)
                                CNN.add(MaxPooling2D(pool_size=(2, 2)))
                            CNN.add(Flatten())

                            #Dense layers (add dropout for models with additional layers)
                            CNN.add(Dense((int(64*dense_w)), activation='relu'))
                            if dense_l >= 2:
                                CNN.add(Dense((int(32*dense_w)), activation='relu'))
                                if dense_l >= 3: 
                                    CNN.add(Dense((int(16*dense_w)), activation='relu'))
                                    CNN.add(Dropout(dropout_rate))
                                    if dense_l >= 4:
                                        CNN.add(Dense((int(8*dense_w)), activation='relu'))
                            CNN.add(Dense(3, activation='sigmoid'))

                            #Compile and train model
                            learning_rate = 0.001 # set the learning rate value
                            
                            # use the Adam optimizer with the specified learning rate
                            optimizer = Adam(learning_rate=learning_rate)
                            CNN.compile(optimizer=optimizer, loss='categorical_crossentropy', 
                                        metrics=['accuracy'])
                            print("Depth-" + str(depth), "Dropout-" + str(dropout_rate), 
                                  "Kernel-" + str(kernel_size), "Output-" + str(output), 
                                  "DenseW-" + str(dense_w), "DenseL-" + str(dense_l),
                                  "Batch-" + str(batch))
                            early_stopping = EarlyStopping(monitor='val_accuracy', 
                                                           mode='max', patience=5, 
                                                           restore_best_weights=True)
                            history = CNN.fit(X_train, y_train, validation_data=(X_test, y_test), 
                                              epochs=10, batch_size=batch, callbacks=[early_stopping])
                            
                            # Summarize history for accuracy
                            plt.figure()
                            plt.plot(history.history['accuracy'])
                            plt.plot(history.history['val_accuracy'])
                            plt.title('Model accuracy')
                            plt.ylabel('Accuracy')
                            plt.xlabel('Epoch')
                            plt.legend(['Train', 'Validation'], loc='upper left')
                            plt.savefig(fig_path + "/Depth-" + str(depth) + 
                                        "Dropout-" + str(dropout_rate) + 
                                        "Kernel-" + str(kernel_size) + 
                                        "Output-" + str(output) + 
                                        "DenseW-" + str(dense_w) + 
                                        "DenseL-" + str(dense_l) + 
                                        "Batch-" + str(batch) + ".jpg")
                            plt.show()

                            #Show classification alignment with base truth
                            pred = CNN.predict(X_test)
                            count(pred)
                            #AUC score
                            auc_score = roc_auc_score(y_test, pred, multi_class='ovr')
                            print("AUC Score: ", auc_score)
                            y_test_c, pred_c = [y_test.argmax(axis=1),pred.argmax(axis=1)]
                            x = list(range(1, len(y_train)+1))
                            fig = plt.figure(figsize=(20, 5)) # width, height in inches
                            plt.plot(x, pred_c, label='Predicted')
                            plt.plot(x, y_test_c, label='Actual')
                            plt.xlabel('Frame #')
                            plt.ylabel('Label')
                            plt.title('Prediction and Actual Comparison')
                            plt.legend(["Predicted","Actual"])
                            plt.savefig(fig_path + "/Depth-" + str(depth) + 
                                        "Dropout-" + str(dropout_rate) + 
                                        "Kernel-" + str(kernel_size) + 
                                        "Output-" + str(output) + 
                                        "DenseW-" + str(dense_w) + 
                                        "DenseL-" + str(dense_l) + 
                                        "Batch-" + str(batch) + "linegraphs.jpg")
                            plt.show()

In [ ]:
# Given classification, what vertebrae # is the probe over?

pred = CNN.predict(X_train[0:300]).argmax(axis = 1)
y_train.reverse()
pred = y_train[0:100].argmax(axis = 1)
print(pred)

seen_frames = []
state = "Tail"
prev = 2
vcount = 0
icount = 0
for frame in pred: #possible transitions
    #2 space
    if frame == 2:
        state = "Tail"
    #1 space
    elif frame == 1 and prev == 0:
        state = "Vert" + str(vcount+1)
        vcount += 1
        prev = 1
    elif frame == 1:
        pass
    #0 space
    elif frame == 0 and prev == 2:
        state = "Inter1"
        prev = 0
    elif frame == 0 and prev == 1:
        state = "Inter" + str(icount+1)
        icount += 1
        prev = 0
    elif frame == 0 and prev == 0:
        pass
    print(state)
    seen_frames.append(frame)

In [ ]:
#How long does it take to feed data of x # of frames forward through the CNN in real time?

import time
times = []
datas = []
iter = 20
max_size = 1000
for i in range(iter):
    data_size = int((i+1)*(max_size/iter))
    datas.append(data_size)
    print(data_size)
    test = X_test[0:data_size]
    start_time = time.time() #Start timer
    pred = CNN.predict(test)
    end_time = time.time() #End timer
    elapsed_time = end_time - start_time
    times.append(elapsed_time)

plt.plot(datas, times)

plt.xlabel('Data size')
plt.ylabel('Feed forward time')
plt.title('Affect of Data Size on Feedforward Time Through CNN')
plt.show()